In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import sys
import os

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)
sys.path.insert(0, "./")
sys.path.insert(0, "/dadendev/nlp/examples/text_summarization/BertSum/")

In [4]:
print(sys.path)

['/dadendev/nlp/examples/text_summarization/BertSum/', './', '/dadendev/nlp', '/dadendev/anaconda3/envs/cm3/lib/python36.zip', '/dadendev/anaconda3/envs/cm3/lib/python3.6', '/dadendev/anaconda3/envs/cm3/lib/python3.6/lib-dynload', '', '/home/daden/.local/lib/python3.6/site-packages', '/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages', '/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages/pyrouge-0.1.3-py3.6.egg', '/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages/IPython/extensions', '/home/daden/.ipython']


In [5]:
#"""
from utils_nlp.dataset.cnndm import CNNDMSummarization

train_dataset, test_dataset = CNNDMSummarization(top_n=5)

len(train_dataset)

len(test_dataset)

test_dataset[0]
#"""

[nltk_data] Downloading package punkt to /home/daden/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
0lines [00:00, ?lines/s]
0lines [00:00, ?lines/s]
0lines [00:00, ?lines/s]
0lines [00:00, ?lines/s]


[['marseille',
  ',',
  'france',
  '(',
  'cnn',
  ')',
  'the',
  'french',
  'prosecutor',
  'leading',
  'an',
  'investigation',
  'into',
  'the',
  'crash',
  'of',
  'germanwings',
  'flight',
  '9525',
  'insisted',
  'wednesday',
  'that',
  'he',
  'was',
  'not',
  'aware',
  'of',
  'any',
  'video',
  'footage',
  'from',
  'on',
  'board',
  'the',
  'plane',
  '.'],
 ['marseille',
  'prosecutor',
  'brice',
  'robin',
  'told',
  'cnn',
  'that',
  '``',
  'so',
  'far',
  'no',
  'videos',
  'were',
  'used',
  'in',
  'the',
  'crash',
  'investigation',
  '.',
  '``'],
 ['he',
  'added',
  ',',
  '``',
  'a',
  'person',
  'who',
  'has',
  'such',
  'a',
  'video',
  'needs',
  'to',
  'immediately',
  'give',
  'it',
  'to',
  'the',
  'investigators',
  '.',
  '``'],
 ['robin',
  "'s",
  'comments',
  'follow',
  'claims',
  'by',
  'two',
  'magazines',
  ',',
  'german',
  'daily',
  'bild',
  'and',
  'french',
  'paris',
  'match',
  ',',
  'of',
  'a',
  'cel

In [6]:
from utils_nlp.models.transformers.extractive_summarization import ExtSumProcessor, ExtractiveSummarizer

I1026 04:15:41.799360 140372391651136 file_utils.py:39] PyTorch version 1.2.0 available.
I1026 04:15:41.833203 140372391651136 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
I1026 04:15:41.849030 140372391651136 modeling.py:230] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [7]:
processor = ExtSumProcessor()

I1026 04:15:42.053747 140372391651136 tokenization_utils.py:374] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at ./5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


In [8]:
#"""
ext_sum_train = processor.preprocess(list(train_dataset), list(train_dataset.get_target()))

ext_sum_test = processor.preprocess(list(test_dataset), list(test_dataset.get_target()))

#type(ext_sum_train)
#"""

In [9]:
import glob
BERT_DATA_PATH = BERT_DATA_PATH="./bert_data/"
pts = sorted(glob.glob(BERT_DATA_PATH + 'cnndm.train' + '.[0-9]*.pt'))

In [10]:
pts[0:2]

['./bert_data/cnndm.train.0.bert.pt', './bert_data/cnndm.train.1.bert.pt']

In [11]:
import torch
def get_dataset(file_list):
    #random.shuffle(file_list)
    for file in file_list:
        yield torch.load(file)

In [12]:
from bertsum.models.data_loader  import DataIterator
from bertsum.models import  model_builder, data_loader
class Bunch(object):
    """ Class which convert a dictionary to an object """

    def __init__(self, adict):
        self.__dict__.update(adict)
def get_data_loader(data_files, device, is_labeled=False, batch_size=3000):
    """
    Function to get data iterator over a list of data objects.

    Args:
        dataset (list of objects): a list of data objects.
        is_test (bool): it specifies whether the data objects are labeled data.
        batch_size (int): number of tokens per batch.
        
    Returns:
        DataIterator

    """
    args = Bunch({})
    args.use_interval = True
    args.batch_size = batch_size
    data_iter = None
    data_iter  = data_loader.Dataloader(args, get_dataset(data_files), args.batch_size,device, shuffle=False, is_test=is_labeled)
    return data_iter

In [13]:
device = torch.device("cuda:{}".format(0)) 
def train_iter_func():
    return get_data_loader(pts, device, is_labeled=True)

In [14]:
from transformers import PreTrainedModel,  PretrainedConfig, DistilBertModel, BertModel, DistilBertConfig
summarizer = None
summarizer = ExtractiveSummarizer()

I1026 04:15:42.579849 140372391651136 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at ./a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.1ccd1a11c9ff276830e114ea477ea2407100f4a3be7bdc45d37be9e37fa71c7e
I1026 04:15:42.581028 140372391651136 configuration_utils.py:168] Model config {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": null,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "n_heads": 12,
  "n_layers": 6,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torchscript": false,
  "use_bfloat16": false,
  "vocab_size": 30522
}

I1026 04:15:42.699048 140372391651136 modeling_utils.

In [15]:
# notebook parameters
DATA_FOLDER = "./temp"
CACHE_DIR = "./temp"
DEVICE = "cuda"
NUM_EPOCHS = 1
BATCH_SIZE = 64
NUM_GPUS = 1
MAX_LEN = 150

In [ ]:
### from utils_nlp.common.timer import Timer

    summarizer.fit(
            train_iter_func,
            device= DEVICE,
            num_epochs=NUM_EPOCHS,
            batch_size=BATCH_SIZE,
            num_gpus=NUM_GPUS,
            max_steps=1e4,
            learning_rate=1e-3,
            warmup_steps=1e4*0.3,
            verbose=True,
            report_every=100,
        )
  
    


loss: 0.468674, time: 0.35, step 0.000000 out of total 10000.000000
loss: 49.510865, time: 12.12, step 100.000000 out of total 10000.000000
loss: 50.368191, time: 11.75, step 200.000000 out of total 10000.000000
loss: 51.135715, time: 11.72, step 300.000000 out of total 10000.000000
loss: 50.651170, time: 12.08, step 400.000000 out of total 10000.000000
loss: 49.099066, time: 11.69, step 500.000000 out of total 10000.000000
loss: 49.710841, time: 11.70, step 600.000000 out of total 10000.000000
loss: 50.249900, time: 11.67, step 700.000000 out of total 10000.000000
loss: 50.779532, time: 12.02, step 800.000000 out of total 10000.000000
loss: 49.899698, time: 11.74, step 900.000000 out of total 10000.000000
loss: 49.757523, time: 11.90, step 1000.000000 out of total 10000.000000
loss: 49.934256, time: 11.73, step 1100.000000 out of total 10000.000000
loss: 50.296925, time: 12.03, step 1200.000000 out of total 10000.000000
loss: 49.834056, time: 11.74, step 1300.000000 out of total 10000

In [ ]:
import torch
from utils_nlp.models.bert.extractive_text_summarization import get_data_iter
import os

test_dataset=[]
for i in range(0,6):
    filename = os.path.join(BERT_DATA_PATH, "cnndm.test.{0}.bert.pt".format(i))
    test_dataset.extend(torch.load(filename))

In [ ]:
prediction = summarizer.predict(get_data_iter(test_dataset),
                               device=DEVICE,)

In [ ]:
target = [test_dataset[i]['tgt_txt'] for i in range(len(test_dataset))]

In [ ]:
from utils_nlp.eval.evaluate_summarization import get_rouge
rouge_baseline = get_rouge(prediction, target, "./results/")

In [ ]:
prediction[0]

In [ ]:
test_dataset[0]['tgt_txt']